In [2]:
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
import numpy as np
from treelib import Node, Tree
import random
from random import choice
# Rules on PDF
import tqdm
from tqdm import tqdm
import sys





def my_print(s):
    print(s, flush=True)
    

class Move(Enum):
    TOP = 0
    BOTTOM = 1
    LEFT = 2
    RIGHT = 3



BORDER_TILES = np.array(
    [[True, True, True, True, True],
    [True, False, False, False, True],
    [True, False, False, False, True],
    [True, False, False, False, True],
    [True, True, True, True, True]])

def possible_moves(board: np.array, player):
    valid_moves = []
    free_tiles = board == 0
    my_tiles = board == player

    valid_tiles = (free_tiles | my_tiles) & BORDER_TILES
    
    for i in range(5):
        for j in range(5):
            if valid_tiles[i][j]:
                for k in range(4):
                    if not ((i == 0 and k==0) or (i==4 and k==1) or (j == 0 and k==2) or (j == 4 and k==3)):
                        valid_moves.append((i,j,Move(k)))

    return valid_moves


def make_ply(board, ply, player):
    new_board = deepcopy(board)
    
    if ply[2] == Move.TOP:#what if position is l/r
        new_board [1:ply[0]+1,ply[1]] = new_board[0:ply[0],ply[1]]
        new_board[0,ply[1]] = player
    elif ply[2] == Move.BOTTOM:
        new_board [ply[0]:4,ply[1]] = new_board[ply[0]+1:5,ply[1]]
        new_board[4,ply[1]] = player
    elif ply[2] == Move.LEFT:
        new_board[ply[0],1:ply[1]+1] = new_board[ply[0],0:ply[1]]
        new_board[ply[0],0] = player
    else:
        new_board [ply[0],ply[1]:4] = new_board[ply[0],ply[1]+1:5]
        new_board[ply[0],4] = player

    return new_board


def check_winner(board, player) -> int:
    '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
    #reading paper "Quixo is solved emerge from rules that corner case in which player moving his piece create another line of opponents symbol lose"
    #however is really rare evenience

    i_win = False 
    for x in range(board.shape[0]):
        if board[x, 0] != 0:
            if all(board[x, :] == board[x, 0]):
                if player != board[x, 0]:
                    return -player #if last move make complete a row/col/diag of oppo i lose (-player wins)
                i_win = True #check if last move lead me to win 
                
    for y in range(board.shape[1]):
        if board[0, y] != 0:
            if all(board[:, y] == board[0, y]):
                if player != board[0, y]:
                    return -player
                i_win = True
        
    if board[0, 0] != 0:    
        if all([board[x, x] for x in range(board.shape[0])] == board[0, 0]):
            if player != board[0, 0]:
                return -player
            i_win = True

    if board[0, -1] != 0:
        if all([board[x, -1-x] for x in range(board.shape[0])] == board[0, -1]):
            if player != board[0, -1]:
                return -player
            i_win = True
    
    if i_win:
        return player
    
    return 0 #no winner

MAX_INT = 10000  

#turn are naturally swapped when recurring, then in evaluation negative value for those who make win the opponent
def eval_terminal(board:np.array, player:int):
    
    winner = check_winner(board, player)
    
    if winner == player: #current player (next to move) return a good outcome, opponent has to pick max(-returned) -> minimize -> discard this high (for opponent) outcome
        return MAX_INT
    elif winner == -player: #current player (next to move) return a bad outcome, opponent has to pick max(-returned) -> minimize -> keep this low (for opponent) outcome
        return -MAX_INT
    
    return 0
                
POS_SCORES = np.array([
    [3,2,2,2,3],
    [2,3,2,3,2],
    [2,2,4,2,2],
    [2,3,2,3,2],
    [3,2,2,2,3]
])

TOT_SCORES = np.sum(POS_SCORES)
print(TOT_SCORES)

def heuristic_score(board:np.array, player:int, count: (int, int)): #must be efficient
    
    score = 0
   

    #heuristic1
    #board - player set to 0 positon taken by player, count_nonzero counts opponents or not taken position: 
    #assumption: the higher they are the less probably the player can win -> score is -count
    #in this sense player is trying to prevent opponent to "lock" his moves into keep choosing already taken pieces #NOTE: in testing obtained speedup also
    #score -= np.count_nonzero(board - player) / 25 #max=25
    #heuristic2
    #board + player set to 0 positon taken by opponent, count_nonzero counts player or not taken position: 
    #assumption: the higher they are the more probably the player can win -> score is +count (remember that this scoring happens without appending -val)
    #in this sense player is trying to "lock" opponent into moves that keep choosing already taken pieces #NOTE: can slow down since leave more freedom to player -> more branches
    #score += np.count_nonzero(board + player) / 25 #max=25
    #heuristic3 count of row/col/diag of 4
    
    score += (count[0]-count[1]) / 5 #in case of 4 in a row opponent has higher chance of winning: his move and your move

    score += (count[2]-count[3]) / 25
    #heuristic 4 center
    #score += board[2][2] * player / 25
    #heuristic 5 weight possibilities of quixo (use without h1 and h2)
    score += np.sum(board*POS_SCORES*player) / TOT_SCORES
    
    return score



def eval_terminal_3_4(board, player):
    count_p_4 = 0 #count >4 for player
    count_o_4 = 0 #count >4 for opponent
    count_p_3 = 0 #count >4 for player
    count_o_3 = 0 #count >4 for opponent

    oppo_win = False
    for x in range(board.shape[0]):
        n_p = np.count_nonzero(board[x, :] == player)
        if n_p >= 3:
            if n_p >= 4:
                if n_p == 5:
                    return (MAX_INT,None) # last move created a winning position for opponent (current player)
                count_p_4 += 1
            count_p_3 += 1
        else:
            n_o = np.count_nonzero(board[x, :] == -player)
            if n_o >= 3:
                if n_o >= 4:
                    if n_o == 5:
                        oppo_win = True # last move make me win (current player lose)
                    count_o_4 += 1
                count_o_3 += 1

    for y in range(board.shape[1]):
        n_p = np.count_nonzero(board[:, y] == player)
        if n_p >= 3:
            if n_p >= 4:
                if n_p == 5:
                    return (MAX_INT,None)
                count_p_4 += 1
            count_p_3 += 1
        elif not oppo_win:
            n_o = np.count_nonzero(board[:, y] == -player)
            if n_o >= 3:
                if n_o >= 4:
                    if n_o == 5:
                        oppo_win = True
                    count_o_4 += 1
                count_o_3 += 1
    
    diag1 = np.diag(board)
    
    n_p = np.count_nonzero(diag1 == player)
    if n_p >= 3:
        if n_p >= 4:
            if n_p == 5:
                return (MAX_INT,None)
            count_p_4 += 1
        count_p_3 += 1
    elif not oppo_win:
        n_o = np.count_nonzero(diag1 == -player)
        if n_o >= 3:
            if n_o >= 4:
                if n_o == 5:
                    oppo_win = True
                count_o_4 += 1
            count_o_3 += 1
     
    diag2 = np.diag(np.rot90(board))
    
    n_p = np.count_nonzero(diag2 == player)
    if n_p >= 3:
        if n_p >= 4:
            if n_p == 5:
                return (MAX_INT,None)
            count_p_4 += 1
        count_p_3 += 1
    elif not oppo_win:
        n_o = np.count_nonzero(diag2 == -player)
        if n_o >= 3:
            if n_o >= 4:
                if n_o == 5:
                    oppo_win = True
                count_o_4 += 1
            count_o_3 += 1

    if oppo_win:
        return (-MAX_INT, None)

    return (0,(count_p_4, count_o_4, count_p_3, count_o_3))


def canonical_repr_16simm(board, player): #more compact representation like described in "Quixo is Solved" paper are feasible but more expensive (in term of CPU time)

    new_board = deepcopy(board)
    equiv_board_bytes = [(bytes(new_board),player), (bytes(np.flipud(new_board)),player), (bytes(-new_board),-player), (bytes(np.flipud(-new_board)),-player)]
   
    for _ in range(3):
        new_board = np.rot90(new_board)
        equiv_board_bytes += [(bytes(new_board),player), (bytes(np.flipud(new_board)),player), (bytes(-new_board),-player), (bytes(np.flipud(-new_board)),-player)]
    
    return min(equiv_board_bytes, key= lambda x: x[0])


#NOT USED
def canonical_repr_8simm(board): #more compact representation like described in "Quixo is Solved" paper are feasible but more expensive (in term of CPU time)
    
    new_board = deepcopy(board)
    equiv_board_bytes = [bytes(new_board), bytes(np.flipud(new_board))]
   
    for _ in range(3):
        new_board = np.rot90(new_board)
        equiv_board_bytes += [bytes(new_board), bytes(np.flipud(new_board))]
    
    return min(equiv_board_bytes)

#turn are naturally swapped when recurring

print("ok")

60
ok


In [3]:

MAX_SIZE = 83886160
state_cache = {}

MIN_DEPTH = 3
MAX_DEPTH = 3 #with iterative deepening
THRESHOLD = [-MAX_INT]

#turn are naturally swapped when recurring
def alfabeta(board: np.array, player: int, depth: int, alfa: int, beta: int, max_turn: int, mul_leaf: int , max_depth: int) -> ((int, int, Move), int, bool): #out of object -> faster call

    val, count = eval_terminal_3_4(board, player)
    if val !=0:
        #max d 3 mul leaf = -1 
        #ret at 2: 3 is max receive - val
        #ret at 1: 2 is min receive + val
        #ret at 0: 1 is max receive - val
        #max d 2 mul leaf = 1
        #ret at 1: 2 is max receive - val 
        #ret at 0: 1 is min receive + val
        #max d 1 mul leaf = -1
        #ret at 0: 1 is max receive - val
        return None, mul_leaf * val * (1 - (depth % 2) * 2) , False 
    
    if depth == 0:
        hs = heuristic_score(board, player, count)
        #max d 3 mul leaf = -1 
        #ret at 0: 1 is max receive - hs
        #max d 2 mul leaf = 1
        #ret at 0: 1 is min receive + hs
        #max d 1 mul leaf = -1
        #ret at 0: 1 is max receive - hs
        return None, mul_leaf * hs, False # - hs because hs is evaluation for current node
    
    possible = possible_moves(board, player)
    if len(possible)==0:
        return None, 0, False
    
    is_max = depth%2 == max_turn

    evaluations = []

    for ply in possible:
        new_board = make_ply(board, ply, player)
        
        _, val, _ = alfabeta(new_board, -player , depth-1, alfa, beta, max_turn, mul_leaf, max_depth) #il prossimo layer non riporta valutazione completa in quanto
           
        if is_max:
            alfa = max(alfa, val)
        else:# first state that maximizes outcome (my_outcome = -oppo_outcome (win>loss for oppo) in terminal state) is 2
            beta = min(beta, val)
        

        evaluations.append((ply, val))

        if beta <= alfa:
            break
    
    if is_max:
        best = max(evaluations, key=lambda k: k[1])
    else:
        best = min(evaluations, key=lambda k: k[1])
        
    return best[0], best[1], len(evaluations) != len(possible)


def alfabeta_iter_deep(board, player, min_depth, max_depth):

    t = 0
    for d in range(min_depth, max_depth+1):

        ply, val, _ = alfabeta(board, player, d, -MAX_INT, MAX_INT, d%2, 1 - (d % 2)*2, d)
        if val >= THRESHOLD[t]:
            return ply, val , d
        t += 1 

    assert False, "return val mus be > -MAX_INT"

board_0 = np.zeros((5,5))
win = 0
lose = 0

N_GAMES = 25000 #infinite time if both players plays optimally

my_print("test alfabeta")
custom_bar_format = "{l_bar}{bar:50}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]"
progress_bar = tqdm(range(N_GAMES),dynamic_ncols=True,desc="Game",colour="green",total=N_GAMES,mininterval=0.5,bar_format=custom_bar_format,ncols=100)

for game in progress_bar:
    board = deepcopy(board_0)
    winner = 0
    player = -1
    minmax_turn = ( game % 2 ) * 2 - 1
    random.seed(game+1)

    while winner == 0:
        
        if player == minmax_turn:
            ply, val, max_depth = alfabeta_iter_deep(board, player, MIN_DEPTH, MAX_DEPTH)
        else:
            ply = random.choice(possible_moves(board, player))
        
        board=make_ply(board, ply, player)
        winner=check_winner(board, player)
        
        player = -player

    if winner == minmax_turn:
        win += 1
    elif winner != 0:
        print(f"lose+1 @{game}")
        lose += 1
        print(board, winner, -player, minmax_turn, max_depth, val)

    if game % 2500 == 0:
        print(f"lose @{game}: {lose}")
        
    progress_bar.set_description(f"won: {win}, lost: {lose}, size dict: {sys.getsizeof(state_cache)} bytes") #monitoring slowdown: in 300 game around 1M entries

print(f"won: {win}, lost: {lose}")

my_print("test alfabeta")
custom_bar_format = "{l_bar}{bar:50}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]"
progress_bar = tqdm(range(N_GAMES),dynamic_ncols=True,desc="Game",colour="green",total=N_GAMES,mininterval=0.5,bar_format=custom_bar_format,ncols=100)

for game in progress_bar:
    board = deepcopy(board_0)
    winner = 0
    player = -1
    minmax_turn = ( game % 2 ) * 2 - 1
    random.seed(game)

    while winner == 0:
        
        if player == minmax_turn:
            ply, val, max_depth = alfabeta_iter_deep(board, player, MIN_DEPTH, MAX_DEPTH)
        else:
            ply = random.choice(possible_moves(board, player))
        
        board=make_ply(board, ply, player)
        winner=check_winner(board, player)
        
        player = -player

    if winner == minmax_turn:
        win += 1
    elif winner != 0:
        print(f"lose+1 @{game}")
        lose += 1
        print(board, winner, -player, minmax_turn, max_depth, val)

    if game % 2500 == 0:
        print(f"lose @{game}: {lose}")
        
    progress_bar.set_description(f"won: {win}, lost: {lose}, size dict: {sys.getsizeof(state_cache)} bytes") #monitoring slowdown: in 300 game around 1M entries

print(f"won: {win}, lost: {lose}")


#NOTE: incorrect caching (but doesn't affect too much results) -> win ratio was slightly worse when lot of states were in cache
#caching + heuristic: count4,countXO + alfabeta4: lost 13
#caching + heuristic: count4,countXO + alfabeta4: lost 17
#NOTE: correct caching -> win ratio is slightly better when lot of states are in cache
# caching ME + heuristic: count4,countXO_W,center + alfabeta4: ME = memory efficient, lost (1@2000, 5@5000 ), time: 1h7min #NOTE: correct heuristic, slower 
#25000
# heuristic: count4,countXO_W,center + alfabeta3: lost (0@2500, 3@5000, 4@10000, 5@25000), time: 1h30min #NOTE: correct heuristic, slower 
# caching 1depth + heuristic: count4,countXO_W,center + alfabeta3: lost (1@2500, 7@5000, 17@10000, -@25000), time: 2h #NOTE: correct heuristic, wrong caching (equivalent states) -> stop
# caching 1depth + heuristic: count4,countXO_W,center + alfabeta3: lost (2@2500, 4@5000, 9@10000, 25@25000), time: 1h30min #NOTE: correct heuristic, correct caching (no equivalent states)

#comparison 25000 w/wo caching (seed=game)
# heuristic: count4,countXO_W,center + alfabeta3: lost (1@2500, 3@5000, 5@10000, ?@25000), time: 2h 
# caching 1depth + heuristic: count4,countXO_W,center + alfabeta3: lost (2@2500, 7@5000, 11@10000, 34@25000), time: 1h15min #NOTE CACHE, repr bytes #compare with 1 -> wrong
# caching 1depth + heuristic: count4,countXO_W,center + alfabeta3: lost (2@2500, 7@5000, 11@10000, 34@25000), time: 42min #NOTE CACHE, 7 equiv repr #compare with 1 -> wrong
# heuristic: count4,countXO_W,center + minmax3: lost (1@2500, 3@5000, 5@10000, -@25000), time: 7h #NOTE slowest #TODO, compare with 1 -> wrong
# caching 1depth + heuristic: count4,countXO_W,center + alfabeta3: lost (2@2500, 3@5000, 17@10000, >39@25000), time: 42min #NOTE CACHE, 8 equiv repr #compare with 2,3 -> wrong wrong
# caching 2 + heuristic: count4,countXO_W,center + alfabeta3: lost (1@2500, 3@5000, 5@10000, 10@25000), time: 1h15min #NOTE CACHE
# caching full + heuristic: count4,countXO_W,center + alfabeta3: lost (1@2500, 3@5000, 5@10000, 10@25000), time: 45min  #NOTE CACHE

# caching: full 8s,forward look + heuristic: count4,countXO_W,center + alfabeta3: lost (1@2500, 3@5000, 6@10000, ?@25000), time: 2h #NOTE wrong algorithm
# caching: full 8s,forward look + heuristic: count4,countXO_W_60 + alfabeta3: lost (0@2500, 0@5000, 3@10000, 10@25000), time: 2h #NOTE wrong algorithm
# caching: full 8s,forward look + heuristic: count4_2,countXO_W_72 + alfabeta3: lost (3@2500, 3@5000, 7@10000, >20@25000), time: 2h #NOTE wrong algorithm
# caching: full 8s,forward look,<10M + heuristic: count4,countXO_W_60 + alfabeta3: lost (0@2500, 0@5000, 2@10000, -@25000), time: 2h #NOTE wrong algorithm
#FASTER
# caching: full bytes, forward look + heuristic: count4,countXO_W_66 + alfabeta3: lost (1@2500, 6@5000, -@10000, -@25000), time: 1h #NOTE wrong algorithm
# caching: full bytes, forward look + heuristic: count4,countXO_W_56 + alfabeta3: lost (3@2500, 8@5000, -@10000, -@25000), time: 1h #NOTE wrong algorithm
# caching: full bytes, forward look + heuristic: count4,countXO_W_60 + alfabeta3: lost (0@2500, 0@5000, 3@10000, 10@25000), time: 1h #NOTE wrong algorithm
# caching: full bytes, forward look + heuristic: count4,countXO_W_35 + alfabeta3: lost (1@2500, 1@5000, 4@10000, 10@25000), time: 1h #NOTE wrong algorithm
#keep 35
# caching: full bytes, forward look + heuristic: count4,countXO_W_35,center + alfabeta3: lost (0@2500, 0@5000, 2@10000, 6@25000), time: 1h #NOTE wrong algorithm
# caching: full 8s, forward look + heuristic: count4,countXO_W_35,center + alfabeta3: lost (0@2500, 0@5000, 2@10000, 6@25000), time: 1h #NOTE wrong algorithm
# NOTE: changes: no val swap, min depth = 0
# caching: 8s, same d + heuristic: count4,countXO_W_35,center + alfabeta2: lost (0@2500, 0@5000, 2@10000, 6@25000), time: 1h #NOTE wrong algorithm (check winner)
# caching: 16s, same d + heuristic: count4,countXO_W_35,center + alfabeta2: lost (0@2500, 0@5000, 2@10000, 6@25000), time: 1h #NOTE wrong algorithm (check winner)

# caching: 16s, same d + heuristic: count4/5,countXO_W/60 + alfabeta2: lost (0@2500, 0@5000, 2@10000, 9@25000), time: 1h #NOTE correct algorithmm
# lose always due to move of ai that bring opponent to complete row/col -> introduce count of 3
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta2: lost (0@2500, 0@5000, 0@10000, 2@25000), time: 1h10min #NOTE NOTE NOTE NOTE NOTE NOTE NOTE NOTE NOTE NOTE NOTE NOTE : BEST
# caching: 16s, same d + heuristic: count4_3/5_25 + alfabeta2: lost (2@2500, 2@5000, 2@10000, 14@25000), time: 1h #NOTE bad performance
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta3: lost (2@2500, -@5000, -@10000, -@25000), time:7h #NOTE correct algorithmm
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/35 + alfabeta2: lost (0@2500, 2@5000, 4@10000, -@25000), time:7h #NOTE bad performance
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60+center + alfabeta2: lost (?@2500, ?@5000, ?@10000, ?@25000), time: 1h10min #NOTE bad performance

#iterative deepening: NOTE REALLY GOOD
# approx -70% time, THRESHOLD 0.3
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta2/3: lost (0@2500, 0@5000, 0@10000, 6@25000), time: 4h seed = game+1
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta2/3: lost (0@2500, 0@5000, 0@10000, 5@25000), time: 4h seed = game
#MAX_DEPTH caching added
# caching: 16s, same d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta2/3: lost (?@2500, ?@5000, ?@10000, ?@25000), time: 4h seed = game+1

# correct evaluation function terminal
# 50000 test -> caching: 16s full
# caching: 16s,full + heuristic: count4_3/5_25,countXO_W/60 + alfabeta1/2/3: lost (0@2500, 0@5000, 0@10000, 0@25000), time: 4h30min seed = game+1 #NOTE: faster by 10%
# caching: 16s,full d + heuristic: count4_3/5_25,countXO_W/60 + alfabeta1/2/3: lost (0@2500, 0@5000, 0@10000, 0@25000), time: 4h30min seed = game #TODO: faster by 10%, memory error at 175000

test alfabeta


won: 1, lost: 0, size dict: 64 bytes:   0%|                                                  | 1/25000 [00:03<26:33:18]

lose @0: 0


won: 2501, lost: 0, size dict: 64 bytes:  10%|█████                                             | 2501/25000 [7:34:37<27:06:24]   

lose @2500: 0


won: 5001, lost: 0, size dict: 64 bytes:  20%|██████████                                        | 5001/25000 [11:49:50<30:00:56]

lose @5000: 0


won: 7501, lost: 0, size dict: 64 bytes:  30%|███████████████                                   | 7501/25000 [15:26:34<29:31:42]

lose @7500: 0


won: 8697, lost: 0, size dict: 64 bytes:  35%|█████████████████▍                                | 8697/25000 [17:01:30<20:27:53]